In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.0


In [2]:
#loss function
def nll(scores_pos, scores_neg, eta):
    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adagrad(learning_rate=0.1)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5})

model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     max_ent_size=14505, 
                                     max_rel_size=237,
                                     scoring_type='TransE')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt')




Epoch 1/10
29/29 [==============================] - 1s 52ms/step - loss: 13395.2246
Epoch 2/10
29/29 [==============================] - 1s 38ms/step - loss: 12786.1807
Epoch 3/10
29/29 [==============================] - 1s 37ms/step - loss: 12509.6895
Epoch 4/10
29/29 [==============================] - 1s 37ms/step - loss: 12343.5781
Epoch 5/10
28/29 [===========================>..] - ETA: 0s - loss: 12299.9727
9 triples containing invalid keys skipped!
29/29 [==============================] - 6s 219ms/step - loss: 12230.4512 - val_mrr: 0.0915 - val_mr: 470.8200 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2517
Epoch 6/10
29/29 [==============================] - 1s 37ms/step - loss: 12145.6768
Epoch 7/10
29/29 [==============================] - 1s 39ms/step - loss: 12080.4492
Epoch 8/10
29/29 [==============================] - 1s 38ms/step - loss: 12027.0420
Epoch 9/10
29/29 [==============================] - 1s 37ms/step - loss: 11982.5723
Epoch 10/10
28/29 [===========================>.

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 93s 454ms/step
Time taken: 101.26544547080994


(329.3016929249437,
 0.20396551111124014,
 0.12819258244446619,
 0.35485370388492027)

# Training/eval with partition

In [5]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)


partitioner = PARTITION_ALGO_REGISTRY.get('Bucket')(dataset_loader, k=3)

_split: memory before: 896.0Bytes, after: 13.12MB, consumed: 13.119MB; exec time: 211.03s


In [9]:
optim = tf.optimizers.Adam(learning_rate=0.1, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=9437, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

#model.compile(optimizer='rmsprop', loss=nll)
partitioned_model.compile(optimizer=optim, loss=nll)
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')
start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,
                     epochs=10, callbacks=[checkpoint])
print((time.time()-start))

Epoch 1/10
    277/Unknown - 11s 38ms/step - loss: 1601.5470
Epoch 00001: loss improved from inf to 1601.54700, saving model to ./chkpt1
277/277 [==============================] - 11s 40ms/step - loss: 1601.5470
Epoch 2/10
276/277 [============================>.] - ETA: 0s - loss: 1235.1089
Epoch 00002: loss improved from 1601.54700 to 1233.28357, saving model to ./chkpt1
277/277 [==============================] - 9s 32ms/step - loss: 1233.2836
Epoch 3/10
276/277 [============================>.] - ETA: 0s - loss: 1061.9808
Epoch 00003: loss improved from 1233.28357 to 1060.90527, saving model to ./chkpt1
277/277 [==============================] - 9s 34ms/step - loss: 1060.9053
Epoch 4/10
276/277 [============================>.] - ETA: 0s - loss: 963.2993
Epoch 00004: loss improved from 1060.90527 to 962.56677, saving model to ./chkpt1
277/277 [==============================] - 9s 34ms/step - loss: 962.5668
Epoch 5/10
276/277 [============================>.] - ETA: 0s - loss: 897.6848
E

In [10]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=partitioned_model.data_handler.get_mapper())

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 123s 2s/step
Time taken: 126.11965680122375


(1309.519975490196, 0.06925623684115728, 0.0, 0.19512254901960785)

In [11]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=200,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
104/104 [==============================] - 711s 7s/step
Time taken: 769.5899913311005


(1181.0907133770427,
 0.13077588574264687,
 0.07300127214013113,
 0.24143751834817498)